 Data Processing

In [272]:
# import pandas as pd
# import glob

# # Path to the directory containing CSV files
# file_path_pattern = 'D:/guvi/car_dheko_price_prediction/car_dheko_price_pridiction/*.csv'

# # List all CSV files
# csv_files = glob.glob(file_path_pattern)

# # Read and concatenate CSV files
# dfs = [pd.read_csv(file) for file in csv_files]
# combined_df = pd.concat(dfs, ignore_index=True)

# # Reset index
# combined_df.reset_index(drop=True, inplace=True)

# # Save the combined DataFrame to a new CSV file
# combined_df.to_csv('combined_file.csv', index=False)

# print("CSV files combined successfully!")


In [305]:
import pandas as pd
import re

pd.set_option('display.max_columns', None)

df = pd.read_csv("combined_file.csv")

# print(df.shape)
# print(df.info())

C:\Users\New\AppData\Local\Temp\ipykernel_24084\1758945192.py:6: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("combined_file.csv")


cleaning & null imputation

In [306]:
# extract only numbers
df['km'] = df['km'].str.replace(",","")
df['km'] = pd.to_numeric(df['km'])

#cargo_volumn
df['cargo_volumn'] = df['cargo_volumn'].astype(str)
df['cargo_volumn'] = df['cargo_volumn'].str.extract(r'(\d+)')  
df['cargo_volumn'] = pd.to_numeric(df['cargo_volumn'])  
df['cargo_volumn'] = df['cargo_volumn'].fillna(df['cargo_volumn'].mean()).astype(int) 

#price convert to int 
def convert_to_int(value):
    if pd.isna(value):
        return None  # or `float('nan')` if you prefer

    value = value.replace('₹', '').strip()
    
    # Handle 'Lakh' and 'Crore'
    if 'Lakh' in value:
        value = value.replace('Lakh', '').strip()
        multiplier = 1e5  
    elif 'Crore' in value:
        value = value.replace('Crore', '').strip()
        multiplier = 1e7 
    else:
        multiplier = 1  

    # Remove any commas
    value = value.replace(',', '')

    try:
        # Convert to float and then to int
        return int(float(value) * multiplier)
    except ValueError:
        return None  # or `float('nan')` if you prefer


# Apply the function to the 'price' column
df['price'] = df['price'].apply(convert_to_int)


In [307]:
# List of columns to keep
columns_to_keep = [
    'city','ft', 'bt', 'km', 'transmission', 'ownerno', 'model', 
    'modelyear', 'color', 
    'cargo_volumn' , 'price'
]

# Creating a new DataFrame with only the specified columns
df_filtered = df[columns_to_keep]


In [276]:
# from sklearn.impute import KNNImputer
# import numpy as np

# df = pd.DataFrame({
#     'A': [1, 2, np.nan, 4, 5],
#     'B': [5, np.nan, 7, 8, 10]
# })

# imputer = KNNImputer(n_neighbors=2)
# df_imputed = imputer.fit_transform(df)

# print(pd.DataFrame(df_imputed, columns=df.columns))


In [308]:
df_filtered.head(3)

,city,ft,bt,km,transmission,ownerno,model,modelyear,color,cargo_volumn,price
0,bangalore,Petrol,Hatchback,120000,Manual,3,Maruti Celerio,2015,White,235,400000
1,bangalore,Petrol,SUV,32706,Manual,2,Ford Ecosport,2018,White,352,811000
2,bangalore,Petrol,Hatchback,11949,Manual,1,Tata Tiago,2018,Red,242,585000


In [309]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8369 entries, 0 to 8368
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   city          8369 non-null   object
 1   ft            8369 non-null   object
 2   bt            8365 non-null   object
 3   km            8369 non-null   int64 
 4   transmission  8369 non-null   object
 5   ownerno       8369 non-null   int64 
 6   model         8369 non-null   object
 7   modelyear     8369 non-null   int64 
 8   color         8366 non-null   object
 9   cargo_volumn  8369 non-null   int32 
 10  price         8369 non-null   int64 
dtypes: int32(1), int64(4), object(6)
memory usage: 686.6+ KB


In [311]:
df_filtered = df_filtered.dropna()

In [312]:
df_filtered.isna().sum()

city            0
ft              0
bt              0
km              0
transmission    0
ownerno         0
model           0
modelyear       0
color           0
cargo_volumn    0
price           0
dtype: int64

In [313]:
df_filtered.describe()

,km,ownerno,modelyear,cargo_volumn,price
count,8.362000e+03,8362.000000,8362.000000,8362.000000,8.362000e+03
mean,5.899777e+04,1.359962,2016.508371,361.718847,9.559904e+05
std,7.407727e+04,0.641666,3.904737,111.921104,1.411652e+06
min,0.000000e+00,0.000000,1985.000000,29.000000,2.016100e+04
25%,3.000000e+04,1.000000,2014.000000,285.000000,3.990000e+05
50%,5.372600e+04,1.000000,2017.000000,361.000000,6.015000e+05
75%,8.000000e+04,2.000000,2019.000000,420.000000,9.500000e+05
max,5.500000e+06,5.000000,2023.000000,1728.000000,4.150000e+07


In [314]:
df_filtered.rename(columns= {'ft':"fuel_type",'bt':"body_type",'km':"km_driven"},inplace=True)

In [315]:
df_filtered.to_csv("cleaned_df.csv")